# TensorFlow and TensorBoard with Regularization



## Purpose

The purpose of this lab is threefold.  

1.   to review using `TensorFlow` and `TensorBoard` for modeling and evaluation with neural networks
2.   to review using data science pipelines and cross-validation with neural networks
3.   to review using `TensorFlow` for neural network regularization

We'll be continuting our investigation of the canonical [Titanic Data Set](https://www.kaggle.com/competitions/titanic/overview) that we began [previously](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

## The Titanic

### The Titanic and it's data



RMS Titanic was a British passenger liner built by Harland and Wolf and operated by the White Star Line. It sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after striking an iceberg during her maiden voyage from Southampton, England to New York City, USA.

Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters. 

Though there were about 2,224 passengers and crew members, we are given data of about 1,300 passengers. Out of these 1,300 passengers details, about 900 data is used for training purpose and remaining 400 is used for test purpose. The test data has had the survived column removed and we'll use neural networks to predict whether the passengers in the test data survived or not. Both training and test data are not perfectly clean as we'll see.

Below is a picture of the Titanic Museum in Belfast, Northern Ireland.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/c/c0/Titanic_Belfast_HDR.jpg", width=400, height=400)

### Data Dictionary

*   *Survival* : 0 = No, 1 = Yes
*   *Pclass* : A proxy for socio-economic status (SES)
  *   1st = Upper
  *   2nd = Middle
  *   3rd = Lower
*   *sibsp* : The number of siblings / spouses aboard the Titanic
  *   Sibling = brother, sister, stepbrother, stepsister
  *   Spouse = husband, wife (mistresses and fiancés were ignored)
*   *parch* : The # of parents / children aboard the Titanic
  *   Parent = mother, father
  *   Child = daughter, son, stepdaughter, stepson
  *   Some children travelled only with a nanny, therefore *parch*=0 for them.
*   *Ticket* : Ticket number
*   *Fare* : Passenger fare (British pounds)
*   *Cabin* : Cabin number embarked
*   *Embarked* : Port of Embarkation
  *   C = Cherbourg (now Cherbourg-en-Cotentin), France
  *   Q = Queenstown (now Cobh), Ireland
  *   S = Southampton, England
*   *Name*, *Sex*, *Age* (years) are all self-explanatory

## Libraries and the Data



### Importing libraries

In [2]:
# Load the germane libraries

import pandas as pd
import numpy as np
import seaborn as sns 
from pandas._libs.tslibs import timestamps
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras 
from keras import models
from sklearn.impute import SimpleImputer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.losses import binary_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier

# Load the TensorBoard notebook extension and related libraries
%load_ext tensorboard
import datetime

### Loading the data

In [3]:
# Load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# We need to do this for when we mamke our predictions from the test data at the end
ids = test[['PassengerId']]

## EDA and Preprocessing

### Exploratory Data Analysis

You have already performed EDA on this data set. Look back on what you did before or see [here](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

Of course, feel free to re-run what you have done before or try out some other EDA as you find useful.

### Preprocessing

Let's do the same prepricessing as before.

In [4]:
# Performing preprocessing on the train and test data will be more effecient if we combine the two date sets.
combined = pd.concat([train, test], axis=0, sort=False)

#Age column
combined['Age'].fillna(combined['Age'].median(),inplace=True) # Age

# Embarked column
combined['Embarked'].fillna(combined['Embarked'].value_counts().index[0], inplace=True) # Embarked
combined['Fare'].fillna(combined['Fare'].median(),inplace=True)

# Class column
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
combined['Pclass'] = combined['Pclass'].map(d) #Pclass

# Making Age into adult (1) and child (0)
combined['Child'] = combined['Age'].apply(lambda age: 1 if age>=18 else 0) 

# Break up the string that has the title and names
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'
combined.groupby('Title').count()

# Replace the French titles with Enlgish
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
    for j in range(len(english_titles)):
        if i == j:
            combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

# Seperate the titles into "major" and "others", the latter would be, e.g., Reverend
major_titles = ['Mr','Mrs','Miss','Master']
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')

#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

# Separating the data back into train and test sets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

# Training
X_train = train.drop(['Survived'],axis=1)
y_train = train['Survived']

# Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

## Neural Network Model

### Building the model

#### Define the model as a pipeline

Let's use the data science pipeline for our neural network model.

As you are now using regularization to guard against high variance, i.e. overfitting the data, in the definition of the model below include *dropout* and/or *l2* regularization. Also, feel free to experiment with different activation functions.

In [8]:
# It will help to define our model in terms of a pipeline
def build_classifier(optimizer):
# insert Sequential and layers here
    classifier = Sequential()
    
    classifier.add(Dense(10, activation='relu', input_dim=X_train.shape[1]))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(1, activation='sigmoid'))
    
    classifier.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])
    return classifier

#### Use grid search to find help you tune the parameters

You can play with optimizers, epochs, and batch sizes. The ones that we're suggesting are not necessarily the best.

In [9]:
# Grid Search
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[10, 20, 50],
                  batch_size=[16, 25, 32])
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

Epoch 1/10
45/45 [==============================] - 0s 777us/step - loss: 0.7018 - accuracy: 0.6376
Epoch 2/10
45/45 [==============================] - 0s 689us/step - loss: 0.6589 - accuracy: 0.6713
Epoch 3/10
45/45 [==============================] - 0s 711us/step - loss: 0.5897 - accuracy: 0.7205
Epoch 4/10
45/45 [==============================] - 0s 778us/step - loss: 0.5678 - accuracy: 0.7346
Epoch 5/10
45/45 [==============================] - 0s 600us/step - loss: 0.5313 - accuracy: 0.7458
Epoch 6/10
45/45 [==============================] - 0s 756us/step - loss: 0.5292 - accuracy: 0.7612
Epoch 7/10
45/45 [==============================] - 0s 733us/step - loss: 0.5008 - accuracy: 0.7626
Epoch 8/10
45/45 [==============================] - 0s 733us/step - loss: 0.5007 - accuracy: 0.7626
Epoch 9/10
45/45 [==============================] - 0s 800us/step - loss: 0.4865 - accuracy: 0.7865
Epoch 10/10
45/45 [==============================] - 0s 778us/step - loss: 0.4900 - accuracy: 0.7978

45/45 [==============================] - 0s 622us/step - loss: 0.6396 - accuracy: 0.6620
Epoch 5/20
45/45 [==============================] - 0s 644us/step - loss: 0.5881 - accuracy: 0.7097
Epoch 6/20
45/45 [==============================] - 0s 645us/step - loss: 0.5754 - accuracy: 0.7335
Epoch 7/20
45/45 [==============================] - 0s 778us/step - loss: 0.5756 - accuracy: 0.7447
Epoch 8/20
45/45 [==============================] - 0s 689us/step - loss: 0.5186 - accuracy: 0.7644
Epoch 9/20
45/45 [==============================] - 0s 644us/step - loss: 0.5173 - accuracy: 0.7588
Epoch 10/20
45/45 [==============================] - 0s 644us/step - loss: 0.5150 - accuracy: 0.7714
Epoch 11/20
45/45 [==============================] - 0s 600us/step - loss: 0.4989 - accuracy: 0.7770
Epoch 12/20
45/45 [==============================] - 0s 644us/step - loss: 0.5098 - accuracy: 0.7700
Epoch 13/20
45/45 [==============================] - 0s 556us/step - loss: 0.4858 - accuracy: 0.7854
Epoch 1

45/45 [==============================] - 0s 556us/step - loss: 0.6984 - accuracy: 0.6180
Epoch 3/50
45/45 [==============================] - 0s 667us/step - loss: 0.6190 - accuracy: 0.6882
Epoch 4/50
45/45 [==============================] - 0s 533us/step - loss: 0.5891 - accuracy: 0.6924
Epoch 5/50
45/45 [==============================] - 0s 556us/step - loss: 0.5516 - accuracy: 0.7402
Epoch 6/50
45/45 [==============================] - 0s 578us/step - loss: 0.5483 - accuracy: 0.7360
Epoch 7/50
45/45 [==============================] - 0s 733us/step - loss: 0.5306 - accuracy: 0.7500
Epoch 8/50
45/45 [==============================] - 0s 555us/step - loss: 0.5105 - accuracy: 0.7654
Epoch 9/50
45/45 [==============================] - 0s 533us/step - loss: 0.5080 - accuracy: 0.7683
Epoch 10/50
45/45 [==============================] - 0s 756us/step - loss: 0.5124 - accuracy: 0.7683
Epoch 11/50
45/45 [==============================] - 0s 578us/step - loss: 0.4762 - accuracy: 0.7865
Epoch 12/

45/45 [==============================] - 0s 556us/step - loss: 0.4299 - accuracy: 0.8065
Epoch 34/50
45/45 [==============================] - 0s 688us/step - loss: 0.4442 - accuracy: 0.8050
Epoch 35/50
45/45 [==============================] - 0s 555us/step - loss: 0.4235 - accuracy: 0.8050
Epoch 36/50
45/45 [==============================] - 0s 532us/step - loss: 0.4226 - accuracy: 0.8191
Epoch 37/50
45/45 [==============================] - 0s 555us/step - loss: 0.4179 - accuracy: 0.8149
Epoch 38/50
45/45 [==============================] - 0s 755us/step - loss: 0.4251 - accuracy: 0.8093
Epoch 39/50
45/45 [==============================] - 0s 733us/step - loss: 0.4267 - accuracy: 0.8065
Epoch 40/50
45/45 [==============================] - 0s 578us/step - loss: 0.4150 - accuracy: 0.8275
Epoch 41/50
45/45 [==============================] - 0s 555us/step - loss: 0.4215 - accuracy: 0.8107
Epoch 42/50
45/45 [==============================] - 0s 844us/step - loss: 0.4241 - accuracy: 0.8191
Ep

45/45 [==============================] - 0s 600us/step - loss: 0.4483 - accuracy: 0.7994
Epoch 15/50
45/45 [==============================] - 0s 644us/step - loss: 0.4619 - accuracy: 0.7924
Epoch 16/50
45/45 [==============================] - ETA: 0s - loss: 0.2956 - accuracy: 0.87 - 0s 778us/step - loss: 0.4585 - accuracy: 0.8008
Epoch 17/50
45/45 [==============================] - 0s 578us/step - loss: 0.4356 - accuracy: 0.8121
Epoch 18/50
45/45 [==============================] - 0s 578us/step - loss: 0.4284 - accuracy: 0.8163
Epoch 19/50
45/45 [==============================] - 0s 578us/step - loss: 0.4459 - accuracy: 0.8079
Epoch 20/50
45/45 [==============================] - 0s 577us/step - loss: 0.4436 - accuracy: 0.8191
Epoch 21/50
45/45 [==============================] - 0s 622us/step - loss: 0.4417 - accuracy: 0.8303
Epoch 22/50
45/45 [==============================] - 0s 555us/step - loss: 0.4431 - accuracy: 0.8149
Epoch 23/50
45/45 [==============================] - 0s 577us

...

23/23 [==============================] - 0s 696us/step - loss: 0.5475 - accuracy: 0.7181
Epoch 7/20
23/23 [==============================] - 0s 739us/step - loss: 0.5521 - accuracy: 0.7223
Epoch 8/20
23/23 [==============================] - 0s 870us/step - loss: 0.5200 - accuracy: 0.7532
Epoch 9/20
23/23 [==============================] - 0s 652us/step - loss: 0.5243 - accuracy: 0.7658
Epoch 10/20
23/23 [==============================] - 0s 652us/step - loss: 0.5138 - accuracy: 0.7574
Epoch 11/20
23/23 [==============================] - 0s 652us/step - loss: 0.4926 - accuracy: 0.7938
Epoch 12/20
23/23 [==============================] - 0s 782us/step - loss: 0.5060 - accuracy: 0.7658
Epoch 13/20
23/23 [==============================] - 0s 609us/step - loss: 0.4994 - accuracy: 0.7770
Epoch 14/20
23/23 [==============================] - 0s 652us/step - loss: 0.5016 - accuracy: 0.7728
Epoch 15/20
23/23 [==============================] - 0s 696us/step - loss: 0.5058 - accuracy: 0.7700
Epoch

23/23 [==============================] - 0s 739us/step - loss: 0.5178 - accuracy: 0.7626
Epoch 8/50
23/23 [==============================] - 0s 913us/step - loss: 0.5011 - accuracy: 0.7388
Epoch 9/50
23/23 [==============================] - 0s 913us/step - loss: 0.4974 - accuracy: 0.7809
Epoch 10/50
23/23 [==============================] - 0s 956us/step - loss: 0.4999 - accuracy: 0.7556
Epoch 11/50
23/23 [==============================] - 0s 1ms/step - loss: 0.5005 - accuracy: 0.7683
Epoch 12/50
23/23 [==============================] - 0s 783us/step - loss: 0.5056 - accuracy: 0.7584
Epoch 13/50
23/23 [==============================] - 0s 739us/step - loss: 0.4852 - accuracy: 0.7851
Epoch 14/50
23/23 [==============================] - 0s 739us/step - loss: 0.5011 - accuracy: 0.7584
Epoch 15/50
23/23 [==============================] - 0s 696us/step - loss: 0.4779 - accuracy: 0.7823
Epoch 16/50
23/23 [==============================] - 0s 783us/step - loss: 0.4857 - accuracy: 0.7907
Epoch 

Epoch 39/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4288 - accuracy: 0.8247
Epoch 40/50
23/23 [==============================] - 0s 957us/step - loss: 0.4265 - accuracy: 0.8050
Epoch 41/50
23/23 [==============================] - 0s 695us/step - loss: 0.4052 - accuracy: 0.8303
Epoch 42/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4300 - accuracy: 0.8261
Epoch 43/50
23/23 [==============================] - 0s 695us/step - loss: 0.4211 - accuracy: 0.8022
Epoch 44/50
23/23 [==============================] - 0s 696us/step - loss: 0.4212 - accuracy: 0.8107
Epoch 45/50
23/23 [==============================] - 0s 695us/step - loss: 0.4140 - accuracy: 0.8107
Epoch 46/50
23/23 [==============================] - 0s 696us/step - loss: 0.4235 - accuracy: 0.8050
Epoch 47/50
23/23 [==============================] - 0s 782us/step - loss: 0.4212 - accuracy: 0.8219
Epoch 48/50
23/23 [==============================] - 0s 696us/step - loss: 0.4168 - accuracy: 0

23/23 [==============================] - 0s 739us/step - loss: 0.4565 - accuracy: 0.8275
Epoch 21/50
23/23 [==============================] - 0s 652us/step - loss: 0.4504 - accuracy: 0.8233
Epoch 22/50
23/23 [==============================] - 0s 696us/step - loss: 0.4491 - accuracy: 0.8205
Epoch 23/50
23/23 [==============================] - 0s 652us/step - loss: 0.4510 - accuracy: 0.8121
Epoch 24/50
23/23 [==============================] - 0s 739us/step - loss: 0.4523 - accuracy: 0.8191
Epoch 25/50
23/23 [==============================] - 0s 695us/step - loss: 0.4650 - accuracy: 0.8050
Epoch 26/50
23/23 [==============================] - 0s 1000us/step - loss: 0.4590 - accuracy: 0.8135
Epoch 27/50
23/23 [==============================] - 0s 870us/step - loss: 0.4399 - accuracy: 0.8233
Epoch 28/50
23/23 [==============================] - 0s 652us/step - loss: 0.4472 - accuracy: 0.8121
Epoch 29/50
23/23 [==============================] - 0s 652us/step - loss: 0.4380 - accuracy: 0.8149
E

Epoch 1/50
56/56 [==============================] - 0s 536us/step - loss: 0.8603 - accuracy: 0.5802
Epoch 2/50
56/56 [==============================] - 0s 732us/step - loss: 0.7081 - accuracy: 0.6150
Epoch 3/50
56/56 [==============================] - 0s 571us/step - loss: 0.6623 - accuracy: 0.6375
Epoch 4/50
56/56 [==============================] - 0s 571us/step - loss: 0.6129 - accuracy: 0.6532
Epoch 5/50
56/56 [==============================] - 0s 571us/step - loss: 0.5647 - accuracy: 0.7194
Epoch 6/50
56/56 [==============================] - 0s 553us/step - loss: 0.5351 - accuracy: 0.7598
Epoch 7/50
56/56 [==============================] - 0s 589us/step - loss: 0.5194 - accuracy: 0.7609
Epoch 8/50
56/56 [==============================] - 0s 598us/step - loss: 0.5013 - accuracy: 0.7576
Epoch 9/50
56/56 [==============================] - 0s 536us/step - loss: 0.4990 - accuracy: 0.7688
Epoch 10/50
56/56 [==============================] - 0s 643us/step - loss: 0.4753 - accuracy: 0.7946

#### `TensorBoard`

`TensorBoard` is `TensorFlow`'s visualization toolkit. It is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. The code immediately below will allow us to use TensorBoard.

N.B. When we loaded the libraries, we loaded the TensorBoard notebook extension. (It is the last line of code in the first code chunk.)

In [10]:
# Clear out any prior log data.
!rm -rf logs
# Be careful not to run this command if already have trained your model and you want to use TensorBoard.

# Sets up a timestamped log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_dir)


# The callback function, which will be called in the fit()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fitting the optimal model and evaluating with `TensorBoaard`

Define the early stopping callback. Use your best values from grid serarch with `KerasClassifer` and finally fit the model.

In [12]:
# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='loss', min_delta=1e-8,
                           verbose=1, patience=5,
                           mode='min')

# Using KerasClassifier
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

# Fit the model with the tensorboard_callback
classifier.fit(X_train,
               y_train,
               verbose=1,
               callbacks=[early_stop, tensorboard_callback])


# Warning: If verbose = 0 (silent) or 2 (one line per epoch), then on TensorBoard's Graphs tab there will be an error.
# The other tabs in TensorBoard will still be function, but if you want the graphs then verbose needs to be 1 (progress bar).

Epoch 1/50
56/56 [==============================] - 0s 5ms/step - loss: 0.6927 - accuracy: 0.6498
Epoch 2/50
56/56 [==============================] - 0s 2ms/step - loss: 0.6189 - accuracy: 0.6801
Epoch 3/50
56/56 [==============================] - 0s 1ms/step - loss: 0.5691 - accuracy: 0.7183
Epoch 4/50
56/56 [==============================] - 0s 1ms/step - loss: 0.5522 - accuracy: 0.7340
Epoch 5/50
56/56 [==============================] - 0s 893us/step - loss: 0.5286 - accuracy: 0.7542
Epoch 6/50
56/56 [==============================] - 0s 1ms/step - loss: 0.5289 - accuracy: 0.7621
Epoch 7/50
56/56 [==============================] - 0s 1ms/step - loss: 0.4938 - accuracy: 0.7868
Epoch 8/50
56/56 [==============================] - 0s 875us/step - loss: 0.4944 - accuracy: 0.7856
Epoch 9/50
56/56 [==============================] - 0s 1ms/step - loss: 0.4648 - accuracy: 0.7969
Epoch 10/50
56/56 [==============================] - 0s 2ms/step - loss: 0.4632 - accuracy: 0.7946
Epoch 11/50
56/

In [13]:
# Call TensorBoard within SaturnCloud [Comment this out if you are not in SaturnCloud]
import os
print(f"https://{os.getenv('SATURN_JUPYTER_BASE_DOMAIN')}/proxy/8000/")
%tensorboard --logdir logs/fit --port 8000 --bind_all
# This will generate a hyperlink. Click on that to open TensorBoard!
# (You'll see a 404 error below the link, just ignore that.)

# Call TensorBoard [Not in SaturnCloud]
# Uncomment the next time if you are not in SC
# %tensorboard --logdir logs/fit

https://None/proxy/8000/


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 2176.

#### Results and Predictions

Calculate the predictions, save them as a csv, and print them.

In [14]:
# Your code here (use more cells if you need to)
preds = classifier.predict(test)
results = ids.assign(Survived=preds)
results['Survived'] = results['Survived'].apply(lambda row: 1 if row > 0.5 else 0)
results.to_csv('titanic_submission.csv',index=False)
results.head(20)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


Continue to tweak your model until you are happy with the results based on model evaluation.

## Conclusion

Now that you have the `TensorBoard` to help you look at your model, you can better understand how to tweak your model.

How do your predictions compare to what you did last time?

Remember that your "fancier" model may be less accurate... but that is okay if that is the case since we're trying to guard against variance with regularization techniques.